In [395]:
from dotenv import load_dotenv, find_dotenv
import os
import ssl
env= os.environ
from collections.abc import MutableMapping

In [396]:
from bs4 import BeautifulSoup as bs,CData
import requests
import sys
import re
import hashlib
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import collections.abc
collections.Callable = collections.abc.Callable
# from pymongo import MongoClient
from copy import copy as shallow_copy
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [397]:
ERP_HOMEPAGE_URL = 'https://erp.iitkgp.ac.in/IIT_ERP3/welcome.jsp'
ERP_LOGIN_URL = 'https://erp.iitkgp.ac.in/SSOAdministration/auth.htm'
ERP_SECRET_QUESTION_URL = 'https://erp.iitkgp.ac.in/SSOAdministration/getSecurityQues.htm'
ERP_CDC_MODULE_URL = 'https://erp.iitkgp.ac.in/IIT_ERP3/menulist.htm?module_id=26'
ERP_ACAD_MODULE_URL = 'https://erp.iitkgp.ac.in/IIT_ERP3/menulist.htm?module_id=16'
ERP_TPSTUDENT_URL = 'https://erp.iitkgp.ac.in/TrainingPlacementSSO/TPStudent.jsp'



In [398]:
req_args = {
    'timeout': 20,
    'headers': {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/46.0.2490.86 Safari/537.36',
        'Referer':
        'https://erp.iitkgp.ac.in/SSOAdministration/login.htm?sessionToken=595794DC220159D1CBD10DB69832EF7E.worker3',
    },
    'verify': False
}


In [399]:

class WrongPasswordError(Exception):
    """
    Error which is raised when there no secret question is fetched due to wrong password or other reasons.

    """
    print("Failed to fetch secret question: please check that username and password are valid!")


class SecretAnswerError(Exception):
    """
    Error which is raised when there is issue with user's secret answer settings

    """
    print("Please check your secret answer settings!")



Failed to fetch secret question: please check that username and password are valid!
Please check your secret answer settings!


In [400]:
print("Started erp_login!")

session = requests.Session()

r = session.get(ERP_HOMEPAGE_URL, **req_args)
soup = bs(r.text, 'html.parser')

# print ("Length of the fetched HTML: " + str(len(str(r.text))))
# print (r.text)

Started erp_login!


In [401]:
if soup.find(id='sessionToken'):
    sessionToken = soup.find(id='sessionToken').attrs['value']
else:
    raise Exception("Could not get the sessionToken!")

r = session.post(ERP_SECRET_QUESTION_URL, data={'user_id': env["ERP_USERNAME"]},**req_args)
secret_question = r.text
print(secret_question)

if secret_question is None:
    raise WrongPasswordError

# print ("Secret question from the ERP: " + secret_question)
secret_answer = None
secret_answer_index = None

for i in range(1, 4):
    # print (env['ERP_Q%d'%i])
    if (env['ERP_Q%d'%i] == secret_question):
        secret_answer = env['ERP_A%d'%i]
        secret_answer_index = i
        break
# print(secret_answer_index)
# print(secret_answer)
if secret_answer is None:
    print ('No secret question matched:', secret_question)
    sys.exit(1)

nickname


In [402]:
login_details = {
        'user_id': env['ERP_USERNAME'],
        'password': env['ERP_PASSWORD'],
        'answer': secret_answer,
        'sessionToken': sessionToken,
        'requestedUrl': 'https://erp.iitkgp.ac.in/IIT_ERP3/welcome.jsp',
    }

r = session.post(ERP_LOGIN_URL, data=login_details,
            **req_args)

if len(r.history) < 2:
    print("{answer} (ERP_A{index}) is wrong for {question}".format(
        answer=secret_answer,
        index=secret_answer_index,
        question=secret_question
    ))
    raise SecretAnswerError
ssoToken = re.search(r'\?ssoToken=(.+)$',
                        r.history[1].headers['Location']).group(1)
print ("ERP Login completed!")

ERP Login completed!


In [403]:
r = session.get("https://erp.iitkgp.ac.in/IIT_ERP3/?%s" % ssoToken, **req_args)

In [404]:
NUM_NOTICES_DIFFED=50
ERP_COMPANIES_URL = 'https://erp.iitkgp.ac.in/TrainingPlacementSSO/ERPMonitoring.htm?action=fetchData&jqqueryid=37&_search=false&nd=1448725351715&rows=20&page=1&sidx=&sord=asc&totalrows=50'
ERP_NOTICEBOARD_URL = 'https://erp.iitkgp.ac.in/TrainingPlacementSSO/Notice.jsp'
ERP_NOTICES_URL = 'https://erp.iitkgp.ac.in/TrainingPlacementSSO/ERPMonitoring.htm?action=fetchData&jqqueryid=54&_search=false&nd=1448884994803&rows=20&page=1&sidx=&sord=asc&totalrows=50'
ERP_ATTACHMENT_URL = 'https://erp.iitkgp.ac.in/TrainingPlacementSSO/AdmFilePDF.htm?type=NOTICE&year={}&id={}'
ERP_NOTICE_CONTENT_URL = 'https://erp.iitkgp.ac.in/TrainingPlacementSSO/ShowContent.jsp?year=%s&id=%s'

In [405]:
r = session.get(ERP_NOTICEBOARD_URL, **req_args)
r = session.get(ERP_NOTICES_URL, **req_args)
notices_list = bs(r.text, 'html.parser')
# print ("Total number of notices fetched: %d" % len(notices_list.find_all('row')))

c:\Users\kunal\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [406]:
attachment_args = {
    'timeout': 20,
    'headers': {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_1) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/46.0.2490.86 Safari/537.36',
        'Referer':
        'https://erp.iitkgp.ac.in/SSOAdministration/login.htm?sessionToken=595794DC220159D1CBD10DB69832EF7E.worker3',
        'Cookie': env['COOKIE'],
    },
    'verify': False
}

In [407]:
import requests
url = ERP_ATTACHMENT_URL.format('2022-2023', '14');
print(url)
r = requests.get(url, stream=True, **attachment_args)
print(r.headers)  

https://erp.iitkgp.ac.in/TrainingPlacementSSO/AdmFilePDF.htm?type=NOTICE&year=2022-2023&id=14
{'Date': 'Sun, 10 Jul 2022 23:47:30 GMT', 'Server': 'Apache', 'X-FRAME-OPTIONS': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/pdf'}


In [408]:
notice = {}
notice['attachment_raw'] = b''
for chunk in r.iter_content(4096):
    notice['attachment_raw'] += chunk
# print(notice['attachment_raw'])

In [409]:
import base64
encoded_string = base64.b64encode(notice['attachment_raw'])
print(encoded_string[:10])
r = requests.post('http://localhost:8000/sendfile/',data={'base64string':encoded_string},stream=True)
print(r.text)  

b'JVBERi0xLj'
OK
